![image.png](sts_nasa.png)


# **Science-and-Technology-Society-Use-of-NASA-STELLA-Q2-Spectrometer**

The **Science and Technology Society (STS) of Sarasota-Manatee Counties, Florida** is working with the NASA STELLA (Science and Technology Education for Land/Life Assessment) outreach program as a part of our STEM initiative. According to their site, 

- "NASA STELLA instruments are portable low-cost do-it-yourself (DIY) instruments that support science education, and outreach through scientific engagement, inquiry, and discovery while helping you understand Landsat better". 

**STELLA instruments are developed under the influence and inspiration of Landsat.** This alignment not only fulfills our project needs but also serves as a compelling addition to our STEAM initiatives:

1) To train the minds young Floridians to be more aware of our wetlands, to care for them and about them.  Our program will bring more community publicity to the issue of wetlands change, as well.

2) To expose our middle- and high- school aged students to real science, using real data.  That means how to use instrumentation and understand how the data is collected, and how the data can be used in the real world.  It means not only to create beautiful charts and images that form the good results, but also to understand that data must be collected in a proper and reproducible way, that there physics reasons for lack of accuracy and lack of precision that one must understand and minimize in order to achieve meaningful results.


The NASA STELLA-Q2 is capable of making 18 different spectral measurements from the violet/blue portions of the electromagnetic spectrum out to near infrared regions (beyond our range of vision).The following figure **(1)** shows the visible spectrum by wavelength, and the yellow box indicates the STELLA-Q2 frequency range. 

>![image](Spectrum.png)


More can be found on the STELLA DIY instruments at the following link.

>https://landsat.gsfc.nasa.gov/stella/

The following is a sample-by-sample animation of the type of data acquired from STELLA-Q2 Spectrometer built by STS. STS is providing the python code in a Jupyter Notebook that can be used as an example of how to display the data from the STELLA-Q2 device. We have also provided some sample data to be used with this notebook. It should be noted that we did change the name of some of the headers created from our instrument to add colors to each of the wavelength reading that are made in order to display each wavelength as a corresponding color. The near infrared wavelength readings are colored in grays, wheat and gold where the normal visible spectrum colors are in vivid colors that they represent. 

>
>![image](STELLA_color.gif)
>



### Load Python requirments:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, FloatRangeSlider, Layout


---
---

# **Test : Backyard Grass and Shoreline (at end of data) in continous mode of recording.**

Readings taken in February where much of the grass was brown indicative of winter dormancy. 

![image.png](backyard.jpg)


# **Load STELLA Data and setup this project.**


## 1) Load the Excel data collected on the STELLA-Q2 micro SD card:
---


## 2) Read in only the _First Line_ of average White-Card data for full-sun normailzation:
---

Data Includes:
- STELLA data.xlsx'


## 3) Define colors and labels for each Test Pattern for this project:
---



In [2]:
# 1) read the STELLA data file
file = r'data.xlsx'
df = pd.read_excel(file,index_col=False)



# 2) Define Name and Colors for each unique value in the 'Test' column
test_colors = { 
            'White Card Full Sun':'lightgray',
            'Asphalt':'black',
            'Grass':'limegreen',
            'Shady Grass':'yellow',
            'White Card Shade':'gray',
            'Mud Bank':'brown',
            } 


In [3]:
# Remove leading/trailing whitespaces in column names
df.columns = df.columns.str.strip()

df.head()

,device_type,software_version,UID,Test_number,Test,ImageName,batch,weekday,timestamp_iso8601,decimal_hour,...,irradiance_900nm_wheat_wavelength_nm,irradiance_900nm_wheat_wavelength_uncertainty_nm,irradiance_900nm_wheat_irradiance_uW_per_cm_squared,irradiance_900nm_wheat_irradiance_uncertainty_uW_per_cm_squared,irradiance_940nm_gold_wavelength_nm,irradiance_940nm_gold_wavelength_uncertainty_nm,irradiance_940nm_gold_irradiance_uW_per_cm_squared,irradiance_940nm_gold_irradiance_uncertainty_uW_per_cm_squared,battery_voltage,battery_percent
0,STELLA-Q2,2.4.0,6858,0,White Card Full Sun,./photos/White Card Full Sun.jpg,1,Friday,20240209T192947Z,19.4964,...,900,10,5217.29,626.075,940,10,3449.99,413.999,4.16,98
1,STELLA-Q2,2.4.0,6858,1,White Card Full Sun,./photos/White Card Full Sun.jpg,1,Friday,20240209T192949Z,19.4969,...,900,10,5314.53,637.744,940,10,3517.93,422.152,4.16,98
2,STELLA-Q2,2.4.0,6858,2,White Card Full Sun,./photos/White Card Full Sun.jpg,1,Friday,20240209T192950Z,19.4972,...,900,10,5423.45,650.814,940,10,3606.17,432.740,4.16,98
3,STELLA-Q2,2.4.0,6858,3,White Card Full Sun,./photos/White Card Full Sun.jpg,1,Friday,20240209T192952Z,19.4978,...,900,10,5565.70,667.884,940,10,3707.64,444.917,4.16,98
4,STELLA-Q2,2.4.0,6858,4,White Card Full Sun,./photos/White Card Full Sun.jpg,1,Friday,20240209T192953Z,19.4980,...,900,10,5476.79,657.215,940,10,3658.23,438.987,4.16,98


In [4]:
# Extract all unique wavelengths
wavelengths = sorted(set(re.findall(r'(\d+)nm_', ' '.join(df.columns))))


# **Plot Raw Data Over a Continuous Sample Range:**

In [5]:
# Raw data plots
def update_plot_dashed(timestamp_range):
    plt.figure(figsize=(12, 6))

    start_timestamp_index, end_timestamp_index = timestamp_range
    test = df['Test'][start_timestamp_index]
    testb = df['Test'][start_timestamp_index]
    teste = df['Test'][end_timestamp_index]

    # Loop over the range of timestamp indices
    for timestamp_index in range(start_timestamp_index, end_timestamp_index + 1):
        wavelength_data_list = []
        irradiance_data_list = []

        for wavelength in wavelengths:
            wavelength_pattern = f'{wavelength}nm_(.*?)_irradiance_uW_per_cm_squared'
            wavelength_columns = [col for col in df.columns if re.search(wavelength_pattern, col)]

            for column in wavelength_columns:
                #color = 'black'  # Default color for wavelengths not explicitly defined
                color = test_colors[df['Test'][timestamp_index]]  # Get color based on 'Test' column value


                # Check if the column exists before using it
                if column in df.columns:
                    # Extract wavelength data
                    wavelength_data_str = wavelength
                    try:
                        wavelength_data = int(wavelength_data_str)  # Convert string to integer
                    except ValueError:
                        try:
                            wavelength_data = float(wavelength_data_str)  # Convert string to float
                        except ValueError:
                            continue  # Skip if wavelength cannot be converted to int or float

                    # Append data to lists
                    wavelength_data_list.append(wavelength_data)
                    irradiance_data_list.append(df[column][timestamp_index])

        # Plot data points
        plt.plot(wavelength_data_list, irradiance_data_list, linestyle='-', marker='o', markersize=5, markeredgecolor='blue', mfc='blue', linewidth=3, color=color)

   #  https://www.thoughtco.com/the-visible-light-spectrum-2699036
    # I modified to STELLA Colors too
    plt.axvspan(300, 380, alpha=0.6, color='black',  label = 'Near UV from 300-380nm')
    plt.axvspan(380, 425, alpha=0.9, color='violet', label = 'Violet from 380-425m')
    plt.axvspan(425, 475, alpha=0.6, color='blue',   label = 'Blue from 425-475nm')
    plt.axvspan(475, 510, alpha=0.6, color='cyan',   label = 'Cyan from 475-510nm')
    plt.axvspan(510, 560, alpha=0.6, color='green',  label = 'Green from 510-560nm')
    plt.axvspan(560, 590, alpha=0.2, color='yellow', label = 'Yellow from 560-590nm')
    plt.axvspan(590, 625, alpha=0.3, color='orange', label = 'Orange from 590-625nm')
    plt.axvspan(625, 690, alpha=0.6, color='red',    label = 'Red from 625-690nm')
    plt.axvspan(690, 740, alpha=0.7, color='brown',  label = 'Brwon from 690-740nm')
    plt.axvspan(740, 1000,alpha=0.7, color='black',  label = 'Near IR from 740-1000nm')
 
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Irradiance (uW/cm^2)')
    #plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.xlim(350,950)
    #plt.ylim(0,1500)
    plt.grid()
    #plt.title(f"Raw Readings Irradiance Over Wavelength by Pattern Type: {test} \n \n Test Pattern {df['Test']}")
    plt.title(f"Raw Readings Irradiance Over Wavelength by Pattern Type: {testb} - to -{teste}")
    plt.show()

 


# Create a range slider widget for timestamp indices
timestamp_range_slider = widgets.IntRangeSlider(value=(0, len(df) - 1), min=0, max=len(df) - 1, step=1, description='Timestamp Range', layout=Layout(width='90%'))

# Create an interactive plot using widgets.interactive
interactive_plot = widgets.interactive(update_plot_dashed, timestamp_range=timestamp_range_slider)

# Display the interactive plot
display(interactive_plot)


interactive(children=(IntRangeSlider(value=(0, 166), description='Timestamp Range', layout=Layout(width='90%')…

## Mean Readings for Each Test Pattern:

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import re
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

# Assuming scaling_factors, test_colors, df, and wavelengths are already defined
# Example structure for these variables:
# scaling_factors = {wavelength: value, ...}
# test_colors = {pattern: color, ...}
# df = pd.DataFrame(... with columns for irradiance data ...)
# wavelengths = [list of wavelengths as strings]

def update_plot_mean(timestamp_range):
    plt.figure(figsize=(12, 6))

    start_timestamp_index, end_timestamp_index = timestamp_range
    testb = df['Test'][start_timestamp_index]
    teste = df['Test'][end_timestamp_index]

    # Create a dictionary to store mean irradiance data for each test pattern
    mean_irradiance_data = {}

    # Loop over the range of timestamp indices
    for timestamp_index in range(start_timestamp_index, end_timestamp_index + 1):
        test_pattern = df['Test'][timestamp_index]
        if test_pattern not in mean_irradiance_data:
            mean_irradiance_data[test_pattern] = {}

        for wavelength in wavelengths:
            wavelength_pattern = f'{wavelength}nm_(.*?)_irradiance_uW_per_cm_squared'
            wavelength_columns = [col for col in df.columns if re.search(wavelength_pattern, col)]

            # Apply scaling factor to correct irradiance data
            #scaling_factor = scaling_factors[wavelength]

            for column in wavelength_columns:
                if column in df.columns:
                    # Extract and scale irradiance data
                    irradiance_data = df[column][timestamp_index] 

                    if wavelength not in mean_irradiance_data[test_pattern]:
                        mean_irradiance_data[test_pattern][wavelength] = []

                    mean_irradiance_data[test_pattern][wavelength].append(irradiance_data)

    # Calculate mean irradiance for each wavelength and test pattern
    for test_pattern, irradiance_data in mean_irradiance_data.items():
        wavelength_data_list = []
        mean_irradiance_list = []

        for wavelength, irradiance_values in irradiance_data.items():
            mean_irradiance = np.mean(irradiance_values)
            wavelength_data_list.append(int(wavelength))  # Convert to integer
            mean_irradiance_list.append(mean_irradiance)

        color = test_colors.get(test_pattern, 'black')
        #print(f"Test Pattern: {test_pattern}, Wavelength Data: {wavelength_data_list}, Mean Irradiance: {mean_irradiance_list}")  # Debug statement

        plt.plot(wavelength_data_list, mean_irradiance_list, linestyle='-', linewidth=6, color=color, label=test_pattern)

    # Highlight spectral regions
    plt.axvspan(300, 380, alpha=0.6, color='black')
    plt.axvspan(380, 425, alpha=0.9, color='violet')
    plt.axvspan(425, 475, alpha=0.6, color='blue')
    plt.axvspan(475, 510, alpha=0.6, color='cyan')
    plt.axvspan(510, 560, alpha=0.6, color='green')
    plt.axvspan(560, 590, alpha=0.2, color='yellow')
    plt.axvspan(590, 625, alpha=0.3, color='orange')
    plt.axvspan(625, 690, alpha=0.6, color='red')
    plt.axvspan(690, 740, alpha=0.7, color='brown')
    plt.axvspan(740, 1000, alpha=0.7, color='black')

    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Irradiance (uW/cm^2)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.xlim(400, 950)
    #plt.ylim(0, 10000)
    plt.grid()
    plt.title(f"Mean Raw Readings Irradiance Over Wavelength by Pattern Type: {testb} - {teste}")
    plt.show()

# Create a range slider widget for timestamp indices
timestamp_range_slider = widgets.IntRangeSlider(value=(0, len(df) - 1), min=0, max=len(df) - 1, step=1, description='Timestamp Range', layout=Layout(width='90%'))

# Create an interactive plot using widgets.interactive
interactive_plot = widgets.interactive(update_plot_mean, timestamp_range=timestamp_range_slider)

# Display the interactive plot
display(interactive_plot)


interactive(children=(IntRangeSlider(value=(0, 166), description='Timestamp Range', layout=Layout(width='90%')…